In [1]:
import pandas as pd
import numpy as np

### Area and accuracy estimation for sampling with inclusion probabilities weighted by pixel area, from Tyukavina et al. (in review) "Options for global sampling of geographic data"

#### Example of input data

In [2]:
#Read strata info table with columns:
#"Stratum" - stratum ID, 1 - nstrata;
#"Area_km2" - stratum area in km2 or any other area units, needs to be consistend with pixel size area units in data table;
#"Sample_size" - number of units sampled from each stratum;
#(for sampling without replacement only)"SumPixareaSq" - 
#sum of squared areas of all (not only sampled) units (pixels/polygons) within each stratum
strata = pd.read_csv('4.strata_info.txt', sep = '\t')

In [3]:
strata.head().style.hide_index()

Stratum,Area_km2,Sample_size,SumPixareaSq
1,595255.012800,134,449.495424
2,332992.902600,100,146.067021
3,946369.335100,213,700.634849
4,486272.535600,109,280.287541
5,669855.474600,150,494.909501


In [4]:
#Read sample interpretation table with columns:
#"Stratum" - stratum ID, 1 - nstrata;
#"Pixarea" - area of each sampled pixel/polygon in units consistent with strata area from the sample table (e.g. km2)
#"Map"(for accuracy assessment only) - proportion of target class from the map (0-1) for each sample unit;
#"Reference" - proportion of target class from reference sample classification for each sample unit;
#allowed values are from -1 to 1 for area estimation, and from 0 to 1 for map accuracy assessment.
#(optional)"RefType" - type labels, if the are of target class needs to be estimated separately for multiple sub-types
#(optional)"Correct" - proportion of sample unit (0-1), which is correctly mapped. 
#This column is necessary for maps with more than two classes. 
#For map with two classes it is computed in the function "estimate_OA_two_classes" directly from "Map" and "Reference" columns

data = pd.read_csv('4.Sample_data.txt', sep ='\t')

#Merge data table with sample info table
data = data.merge(strata)
data = data.rename(columns = {'Area_km2':'Ah', 'Sample_size':'nh', 'Pixarea':'ai'})

In [5]:
data.head().style.hide_index()

Stratum,ai,Map,Reference,RefType,Correct,Ah,nh,SumPixareaSq
10,0.000638,1.000000,1.000000,Type1,1.000000,161635.388000,100,112.372396
10,0.000626,1.000000,1.000000,Type1,1.000000,161635.388000,100,112.372396
10,0.000769,1.000000,1.000000,Type1,1.000000,161635.388000,100,112.372396
10,0.000670,1.000000,1.000000,Type1,1.000000,161635.388000,100,112.372396
10,0.000649,1.000000,1.000000,Type1,1.000000,161635.388000,100,112.372396


#### Functions to estimate class area and its standard error

In [6]:
def estimate_area(df: pd.DataFrame) -> float:
    """ 
    Function to estimate target class area from sample refernce values 
    for sampling of units with inclusion probability weighted by the unit area.
    Valid for sampling both with and without replacement.
    ~~~
    Input dataframe with number of lines equal the number of sample pixels/polygons,
    and the following columns:
    "Stratum" (strata IDs 1 - nstrata)
    "Reference" (0-1 - proportion of the sample pixel/polygon identified as target class in reference classification;
    Reference data column could be defined as values in range from -1 to 1 to compute net change area of a target class,
    in this case negative proportions mean net loss, and positive proportions - net gain)
    "Ah" (stratum area, km²<or any other area unit>)
    ~~~
    Returns estimated target class area in units of Ah, 
    negative area in net change computations means overall net loss of a target class
    ~~~
    From Tyukavina et al. (in review) "Options for global sampling of geographic data"
    Appendix, equations 22 and 25
    """
    #Create a copy of column "Stratum" and a new column "ForArea", 
    #computed as wi= yi×Ah (equation 22), where yi == Reference (reference proportion of a target class, estimated for each sample unit)
    df1 = pd.concat([df['Stratum'], pd.Series((df['Reference'] * df['Ah']) , name = 'ForArea')], axis = 1)
    
    ByStratum = df1.groupby(by = ['Stratum'])
    
    # Equation 25, compute area of target class in each stratum
    #Mean area of target class for each stratum
    areastrat = ByStratum.ForArea.sum() / ByStratum.ForArea.count()
    #Sum over all strata
    area = areastrat.sum()

    return area

In [7]:
def estimate_area_SE(df: pd.DataFrame) -> float:
    """ 
    Function to estimate standard error (SE) of the target class area from sample refernce values 
    for sampling of units with inclusion probability weighted by the unit area.
    Valid for sampling both with and without replacement.
    ~~~
    Input dataframe with number of lines equal the number of sample pixels/polygons,
    and the following columns:
    "Stratum" (strata IDs 1 - nstrata)
    "Reference" (0-1 - proportion of the sample pixel/polygon identified as target class in reference classification;
    Reference data column could be defined as values in range from -1 to 1 to compute net change area of a target class,
    in this case negative proportions mean net loss, and positive proportions - net gain)
    "Ah" (stratum area, km²<or any other area unit>)
    ~~~
    Returns estimated SE of the target class area in in units of Ah,
    SE is always a positive number, even if the estimated target class area is negative
    ~~~
    From Tyukavina et al. (in review) "Options for global sampling of geographic data"
    Appendix, equations 22 and 27
    """
    #Create a copy of column "Stratum" and a new column "ForVar", 
    #computed as wi= yi×Ah (equation 22), where yi == Reference (reference proportion of a target class, estimated for each sample unit)
    df1 = pd.concat([df['Stratum'],pd.Series(df['Reference'] * df['Ah'], name = 'ForVar')],axis = 1)
    
    ByStratum = df1.groupby(by = ['Stratum'])
    
    #Equation 27
    #Compute variance for each stratum
    StrataVar = ByStratum.ForVar.var(ddof=1) / ByStratum.ForVar.count() 
    #Sum strata-specific variances
    StrataVarSum = StrataVar.sum()
    #Compute SE of the estimated class area from a sum of strata variances
    SE = np.sqrt(StrataVarSum)
    
    return SE

In [8]:
#Estimate target class area
estimate_area(data)

1223903.8326854417

In [9]:
#Estimate standard error of the target class area
estimate_area_SE(data)

31611.122385173083

In [10]:
#Estimate target class are for each unique type from the column RefType

functions = [estimate_area, estimate_area_SE]
names = ['area, km²','area SE, km²']
results = pd.DataFrame()
datacopy = pd.DataFrame()

for classtype in data['RefType'].unique():
    datacopy = data.copy()
    #A new reference column, where reference values are set to zero if class type is different from the current class type
    datacopy['Reference']=np.where(datacopy['RefType'] == classtype, datacopy['Reference'], 0)
    
    values = {}
    values = {nm:[fn(datacopy)] for fn, nm in zip(functions,names)}
    values["Estimate"] = classtype
    values_pd = pd.DataFrame(values).set_index("Estimate")
    results = pd.concat([values_pd, results])

results = results.reset_index()

results.style.hide_index().format({name: '{:.4f}' for name in names})


Estimate,"area, km²","area SE, km²"
Type3,383052.9482,32929.1133
Type2,608160.8876,38476.2797
Type0,0.0000,0.0000
Type1,232689.9969,27599.9173


#### Functions to estimate overall map area and its standard error

In [11]:
def estimate_OA_two_classes (df: pd.DataFrame) -> float:
    """ 
    Function to estimate overall accuracy of the map that has two classes (target class vs. no target class)
    for sampling of units with inclusion probability weighted by the unit area.
    Valid for sampling both with and without replacement.
    ~~~
    Input dataframe with number of lines equal the number of sample pixels/polygons,
    ans the following columns:
    "Stratum" (strata IDs 1 - nstrata)
    "Map" (0-1 - proportion of the sample pixel/polygon identified as target class in the map)
    "Reference" (0-1 - proportion of the sample pixel/polygon identified as target class in reference classification)
    "Ah" (stratum area, km²<or any other area unit>)
    ~~~
    Returns estimated overall accuracy of the map expressed as percent of the total study area
    ~~~
    From Tyukavina et al. (in review) "Options for global sampling of geographic data"
    Appendix, equations 22, 25 and 26
    """
    #Create a copy of columns "Stratum","Ah" and a new column "Correct, 
    #where the proportion of the sample unit that is correctly classified (as either of map classes) is computed as: 
    #min ("Map", "Reference") + min (1-"Map", 1-"Reference"), and multiplied by stratum area (equation 22)
    df1 = pd.concat([df['Stratum'], df['Ah'],
                     pd.Series((np.minimum(df['Map'],df['Reference']) + np.minimum((1 - df['Map']),(1 - df['Reference']))) * df['Ah'],name = 'Correct')], axis = 1)
                     
    ByStratum = df1.groupby(by = ['Stratum'])
    
    #Equation 25
    #compute correctly classified area in each stratum
    CorrectlyClassified = ByStratum.Correct.sum() / ByStratum.Correct.count()
    #sum correctly classified areas accross all strata
    CorrectlyClassifiedSum = CorrectlyClassified.sum()
    
    #Equation 26
    #Derive an Overall Accuracy metric by dividing the sum of correctly classified areas by the total study area
    OA = (CorrectlyClassifiedSum / ((ByStratum.Ah.median()).sum())) * 100

    return OA

In [12]:
estimate_OA_two_classes(data)

92.08916693742641

In [13]:
def estimate_OA_multiple_classes (df: pd.DataFrame) -> float:
    """ 
    Function to estimate overall accuracy of the map that has multiple classes
    for sampling of units with inclusion probability weighted by the unit area.
    Valid for sampling both with and without replacement.
    ~~~
    Input dataframe with number of lines equal the number of sample pixels/polygons,
    ans the following columns:
    "Stratum" (strata IDs 1 - nstrata)
    "Correct" - proportion of sample unit (0-1), which is correctly mapped
    "Ah" (stratum area, km²<or any other area unit>)
    ~~~
    Returns estimated overall accuracy of the map expressed as percent of the total study area
    ~~~
    From Tyukavina et al. (in review) "Options for global sampling of geographic data"
    Appendix, equations 22, 25 and 26
    """
    #Create a copy of columns "Stratum","Ah" and a new column "Correct", 
    #where the proportion of the sample unit that is correctly classified is multiplied by stratum area (equation 22)
    df1 = pd.concat([df['Stratum'], df['Ah'],
                     pd.Series(df['Correct'] * df['Ah'],name = 'Correct')], axis = 1)
                     
    ByStratum = df1.groupby(by = ['Stratum'])
    
    #Equation 25
    #compute correctly classified area in each stratum
    CorrectlyClassified = ByStratum.Correct.sum() / ByStratum.Correct.count()
    #sum correctly classified areas accross all strata
    CorrectlyClassifiedSum = CorrectlyClassified.sum()
    
    #Equation 26
    #Derive an overall accuracy metric by dividing the sum of correctly classified areas by the total study area
    OA = (CorrectlyClassifiedSum / ((ByStratum.Ah.median()).sum())) * 100

    return OA

In [14]:
estimate_OA_multiple_classes(data)

92.08916693742641

In [15]:
def estimate_SE_OA_two_classes (df: pd.DataFrame) -> float:
    """ 
    Function to estimate the standard error of overall accuracy of the map that has two classes 
    (target class vs. no target class) for sampling of units with inclusion probability weighted by the unit area.
    Valid for sampling both with and without replacement.
    ~~~
    Input dataframe with number of lines equal the number of sample pixels/polygons,
    ans the following columns:
    "Stratum" (strata IDs 1 - nstrata)
    "Map" (0-1 - proportion of the sample pixel/polygon identified as target class in the map)
    "Reference" (0-1 - proportion of the sample pixel/polygon identified as target class in reference classification)
    "Ah" (stratum area, km²<or any other area unit>)
    ~~~
    Returns estimated standard error of the overall accuracy of the map expressed as percent of the total study area
    ~~~
    From Tyukavina et al. (in review) "Options for global sampling of geographic data"
    Appendix, equations 22, 27 and 28
    """
    #Create a copy of columns "Stratum","Ah" and a new column "Correct, 
    #where the proportion of the sample unit that is correctly classified (as either of map classes) is computed as: 
    #min ("Map", "Reference") + min (1-"Map", 1-"Reference"), and multiplied by stratum area (equation 22)
    df1 = pd.concat([df['Stratum'], df['Ah'],
                     pd.Series((np.minimum(df['Map'],df['Reference']) + np.minimum((1 - df['Map']),(1 - df['Reference']))) * df['Ah'],name = 'Correct')], axis = 1)
                     
    ByStratum = df1.groupby(by = ['Stratum'])
    
    #Equation 27
    #compute variance for each stratum
    StrataVar = ByStratum.Correct.var(ddof=1) / ByStratum.Correct.count()
    #sum strata-specific variances
    StrataVarSum = StrataVar.sum()
    
    #Compute SE of the estimated class area from a sum of strata variances divided by the total study area
    SE = (np.sqrt(StrataVarSum) / ((ByStratum.Ah.median()).sum())) * 100
    
    return SE

In [16]:
estimate_SE_OA_two_classes(data)

0.7080373503205022

In [17]:
def estimate_SE_OA_multiple_classes (df: pd.DataFrame) -> float:
    """ 
    Function to estimate the standard error of overall accuracy of the map that has multiple classes
    for sampling of units with inclusion probability weighted by the unit area.
    Valid for sampling both with and without replacement.
    ~~~
    Input dataframe with number of lines equal the number of sample pixels/polygons,
    ans the following columns:
    "Stratum" (strata IDs 1 - nstrata)
    "Correct" - proportion of sample unit (0-1), which is correctly mapped
    "Ah" (stratum area, km²<or any other area unit>)
    ~~~
    Returns estimated standard error of the overall accuracy of the map expressed as percent of the total study area
    ~~~
    From Tyukavina et al. (in review) "Options for global sampling of geographic data"
    Appendix, equations 22, 27 and 28
    """
    #Create a copy of columns "Stratum","Ah" and a new column "Correct", 
    #where the proportion of the sample unit that is correctly classified is multiplied by stratum area (equation 22)
    df1 = pd.concat([df['Stratum'], df['Ah'],
                     pd.Series(df['Correct'] * df['Ah'],name = 'Correct')], axis = 1)
                     
    ByStratum = df1.groupby(by = ['Stratum'])
    
    #Equation 27
    #compute variance for each stratum
    StrataVar = ByStratum.Correct.var(ddof=1) / ByStratum.Correct.count()
    #sum strata-specific variances
    StrataVarSum = StrataVar.sum()
    
    #Compute SE of the estimated class area from a sum of strata variances divided by the total study area
    SE = (np.sqrt(StrataVarSum) / ((ByStratum.Ah.median()).sum())) * 100
    
    return SE

In [18]:
estimate_SE_OA_multiple_classes(data)

0.7080373503205022

In [19]:
def estimate_UA (df: pd.DataFrame) -> float:
    """ 
    Function to estimate user's accuracy of target class
    for sampling of units with inclusion probability weighted by the unit area.
    Valid for sampling both with and without replacement.
    ~~~
    Input dataframe with number of lines equal the number of sample pixels/polygons,
    and the following columns:
    "Map" (0-1 - proportion of the sample pixel/polygon identified as target class in the map)
    "Reference" (0-1 - proportion of the sample pixel/polygon identified as target class in reference classification)
    "Ah" (stratum area, km²<or any other area unit>)
    "nh" (number of sample pixels in stratum h)
    "ai" (area of sampled pixel in the same units as Ah)
    ~~~
    Returns estimated user's accuracy of target class expressed as percentage
    ~~~
    From Tyukavina et al. (in review) "Options for global sampling of geographic data"
    Appendix, equations 21, 23, 29, 30, 31
    """
    #Create a copy of columns "nh","Ah","ai" and new columns computed as (equation 23):
    
    #"Correct", the proportion of the sample unit that is correctly classified as target class:
    # min ("Map", "Reference"), and multiplied by the sample unit area; 
    #"Mapped": "Map", multiplied by the sample unit area.
    df1 = pd.concat([df['nh'],df['Ah'],df['ai'], pd.Series((np.minimum(df['Map'],df['Reference']) * df['ai']) , name = 'Correct'),
                    pd.Series((df['Map'] * df['ai']) , name = 'Mapped')],axis = 1)
    
    #Equation 21 Compute inclusion probability for each sampled unit
    df1['PIi'] = df1['nh'] * df1['ai'] / df1['Ah']
    
    #Equation 30
    West = (df1.Correct / df1.PIi).sum()
    #Equation 31
    Zest = (df1.Mapped / df1.PIi).sum()
    
    #Equation 29
    UA = West / Zest * 100

    return UA

In [20]:
estimate_UA(data)

80.6910655161679

In [21]:
def estimate_PA (df: pd.DataFrame) -> float:
    """ 
    Function to estimate producer's accuracy of target class
    for sampling of units with inclusion probability weighted by the unit area.
    Valid for sampling both with and without replacement.
    ~~~ 
    Input dataframe with number of lines equal the number of sample pixels/polygons,
    and the following columns:
    "Map" (0-1 - proportion of the sample pixel/polygon identified as target class in the map)
    "Reference" (0-1 - proportion of the sample pixel/polygon identified as target class in reference classification)
    "Ah" (stratum area, km²<or any other area unit>)
    "nh" (number of sample pixels in stratum h)
    "ai" (area of sampled pixel in the same units as Ah)
    ~~~
    Returns estimated producer's accuracy of target class expressed as percentage
    ~~~
    From Tyukavina et al. (in review) "Options for global sampling of geographic data"
    Appendix, equations 21, 23, 29-31
    """
    #Create a copy of columns "nh", "Ah","ai" and new columns computed as (equation 23):
    #"Correct", the proportion of the sample unit that is correctly classified as target class:
    # min ("Map", "Reference"), and multiplied by the sample unit area;
    #"Ref": "Reference", multiplied by the sample unit area.
    df1 = pd.concat([df['nh'],df['Ah'],df['ai'], pd.Series((np.minimum(df['Map'],df['Reference']) * df['ai']), name = 'Correct'),
                    pd.Series((df['Reference'] * df['ai']) , name = 'Ref')],axis = 1)
    
    #Equation 21 Compute inclusion probability for each sampled pixel
    df1['PIi'] = df1['nh'] * df1['ai'] / df1['Ah']
    
    #Equation 30
    West = (df1.Correct / df1.PIi).sum()
    #Equation 31
    Zest = (df1.Ref / df1.PIi).sum()
    
    #Equation 29
    PA = West / Zest * 100

    return PA

In [22]:
estimate_PA(data)

93.62678633355188

In [23]:
def estimate_UA_SE_with_replacement(df: pd.DataFrame) -> float:
    """ 
    Function to estimate the standard error (SE) of user's accuracy of target class
    for sampling of units with inclusion probability weighted by the unit area.
    Valid for sampling with replacement.
    ~~~
    Input dataframe with number of lines equal the number of sample pixels/polygons,
    and the following columns:
    "Map" (0-1 - proportion of the sample pixel/polygon identified as target class in the map)
    "Reference" (0-1 - proportion of the sample pixel/polygon identified as target class in reference classification)
    "Ah" (stratum area, km²<or any other area unit>)
    "nh" (number of sample pixels in stratum h)
    "ai" (area of sampled pixel in the same units as Ah)
    ~~~
    Returns estimated SE of user's accuracy of target class expressed as percentage
    ~~~
    From Tyukavina et al. (in review) "Options for global sampling of geographic data"
    Appendix, equations 21, 23, 29-32
    """
    #Create a copy of columns "nh", "Ah","ai" and new columns computed as (equation 23):
    
    #"Correct", the proportion of the sample unit that is correctly classified as target class:
    # min ("Map", "Reference"), and multiplied by the sample unit area; 
    #"Mapped": "Map", multiplied by the sample unit area.
    df1 = pd.concat([df['nh'],df['Ah'],df['ai'], pd.Series((np.minimum(df['Map'],df['Reference']) * df['ai']) , name = 'Correct'),
                    pd.Series((df['Map'] * df['ai']) , name = 'Mapped')],axis = 1)
    
    #Equation 21 Compute inclusion probability for each sampled unit
    df1['PIi'] = df1['nh'] * df1['ai'] / df1['Ah']
    
    #Equation 30
    West = (df1.Correct / df1.PIi).sum()
    #Equation 31
    Zest = (df1.Mapped / df1.PIi).sum()
    #Equation 29
    UA = West / Zest
    
    # Equation 32
    var_i = (1 - df1['PIi']) * (df1['Correct'] - UA * df1['Mapped']) * (df1['Correct'] - UA * df1['Mapped']) / (df1['PIi'] * df1['PIi'])
    var = np.sum(var_i) / Zest**2
    
    SE = np.sqrt(var) * 100
     
    return SE

In [24]:
estimate_UA_SE_with_replacement(data)

1.8753281165637898

In [25]:
def estimate_UA_SE_without_replacement (df: pd.DataFrame) -> float:
    """ 
    Function to estimate the standard error (SE) of user's accuracy of target class
    for sampling of units with inclusion probability weighted by the unit area.
    Valid for sampling without replacement.
    ~~~
    Input dataframe with number of lines equal the number of sample pixels/polygons,
    and the following columns:
    "Stratum" (strata IDs 1 - nstrata)
    "Map" (0-1 - proportion of the sample pixel/polygon identified as target class in the map)
    "Reference" (0-1 - proportion of the sample pixel/polygon identified as target class in reference classification)
    "Ah" (stratum area, km²<or any other area unit>)
    "nh" (number of sample pixels in stratum h)
    "ai" (area of sampled pixel in the same units as Ah)
    "SumPixareaSq" (sum of squared areas of all (not only sampled) units (pixels/polygons) within each stratum)
    ~~~
    Returns estimated SE of user's accuracy of target class expressed as percentage
    ~~~
    From Tyukavina et al. (in review) "Options for global sampling of geographic data"
    Appendix, equations 21, 23, 29-31, 33-35
    """
    #Create a copy of columns "nh", "Ah","ai", "SumPixareaSq" and new columns computed as (equation 23):
    
    #"Correct", the proportion of the sample unit that is correctly classified as target class:
    # min ("Map", "Reference"), and multiplied by the sample unit area; 
    #"Mapped": "Map", multiplied by the sample unit area.
    df1 = pd.concat([df['Stratum'],df['nh'],df['Ah'],df['ai'],df['SumPixareaSq'], pd.Series((np.minimum(df['Map'],df['Reference']) * df['ai']) , name = 'Correct'),
                    pd.Series((df['Map'] * df['ai']) , name = 'Mapped')],axis = 1)
    
    #Equation 21 Compute inclusion probability for each sampled unit
    df1['PIi'] = df1['nh'] * df1['ai'] / df1['Ah']
    
    #Equation 30
    West = (df1.Correct / df1.PIi).sum()
    #Equation 31
    Zest = (df1.Mapped / df1.PIi).sum()
    #Equation 29
    UA = West / Zest
    
    #Equation 35
    ByStratum = df1.groupby(by = ['Stratum'])
    K = (ByStratum.SumPixareaSq.median() * ByStratum.nh.median()) / (ByStratum.Ah.median())**2
    
    n = len(df1)
    
    PIi = np.broadcast_to(df1.PIi, (n, n))
    PIj = PIi.T
    Stratumh = np.broadcast_to(df1.Stratum, (n, n))
    nh = np.broadcast_to(df1.nh, (n, n)) 
    Kh = np.broadcast_to(K[df1['Stratum']], (n, n))
    PIiPIj = PIi * PIj
    samestratum = (Stratumh == Stratumh.T).astype(int) #to check whether pixels i and j are in the same stratum
    
    # Equation 34
    PIij_num = (nh - 1) * PIiPIj
    PIij_denom = nh - PIi - PIj + Kh
    PIij = PIij_num / PIij_denom
    
    # Equation 33
    var_i = (df1['Correct'] - UA * df1['Mapped']) / df1['PIi']
    var_ij = (1 - PIiPIj / PIij) * np.tensordot(var_i, var_i, axes=0) * samestratum
    var_ij[np.diag_indices_from(var_ij)] = (1 - df1['PIi']) * var_i ** 2
    var = np.sum(var_ij) / Zest ** 2
    
    SE = np.sqrt(var)*100
     
    return SE

In [26]:
estimate_UA_SE_without_replacement(data)

1.8257850328247602

In [27]:
def estimate_PA_SE_with_replacement(df: pd.DataFrame) -> float:
    """ 
    Function to estimate the standard error (SE) of producer's accuracy of target class
    for sampling of units with inclusion probability weighted by the unit area.
    Valid for sampling with replacement.
    ~~~
    Input dataframe with number of lines equal the number of sample pixels/polygons,
    and the following columns:
    "Map" (0-1 - proportion of the sample pixel/polygon identified as target class in the map)
    "Reference" (0-1 - proportion of the sample pixel/polygon identified as target class in reference classification)
    "Ah" (stratum area, km²<or any other area unit>)
    "nh" (number of sample pixels in stratum h)
    "ai" (area of sampled pixel in the same units as Ah)
    ~~~
    Returns estimated SE of producer's accuracy of target class expressed as percentage
    ~~~
    From Tyukavina et al. (in review) "Options for global sampling of geographic data"
    Appendix, equations 21, 23, 29-32
    """
    #Create a copy of columns "nh", "Ah","ai" and new columns computed as (equation 23):
    #"Correct", the proportion of the sample unit that is correctly classified as target class:
    # min ("Map", "Reference"), and multiplied by the sample unit area;
    #"Ref": "Reference", multiplied by the sample unit area.
    df1 = pd.concat([df['nh'],df['Ah'],df['ai'], pd.Series((np.minimum(df['Map'],df['Reference']) * df['ai']), name = 'Correct'),
                    pd.Series((df['Reference'] * df['ai']) , name = 'Ref')],axis = 1)
    
    #Equation 21 Compute inclusion probability for each sampled unit
    df1['PIi'] = df1['nh'] * df1['ai'] / df1['Ah']
    
    #Equation 30
    West = (df1.Correct / df1.PIi).sum()
    #Equation 31
    Zest = (df1.Ref / df1.PIi).sum()
    #Equation 29
    PA = West / Zest
    
    # Equation 32
    var_i = (1 - df1['PIi']) * (df1['Correct'] - PA * df1['Ref'])*(df1['Correct'] - PA * df1['Ref']) / (df1['PIi'] * df1['PIi'])
    var = np.sum(var_i) / Zest**2
    
    SE = np.sqrt(var)*100
     
    return SE

In [28]:
estimate_PA_SE_with_replacement(data)

1.432911087835729

In [29]:
def estimate_PA_SE_without_replacement (df: pd.DataFrame) -> float:
    """ 
    Function to estimate the standard error (SE) of producer's accuracy of target class
    for sampling of units with inclusion probability weighted by the unit area.
    Valid for sampling without replacement.
    ~~~
    Input dataframe with number of lines equal the number of sample pixels/polygons,
    and the following columns:
    "Stratum" (strata IDs 1 - nstrata)
    "Map" (0-1 - proportion of the sample pixel/polygon identified as target class in the map)
    "Reference" (0-1 - proportion of the sample pixel/polygon identified as target class in reference classification)
    "Ah" (stratum area, km²<or any other area unit>)
    "nh" (number of sample pixels in stratum h)
    "ai" (area of sampled pixel in the same units as Ah)
    "SumPixareaSq" (sum of squared areas of all (not only sampled) units (pixels/polygons) within each stratum)
    ~~~
    Returns estimated SE of producer's accuracy of target class expressed as percentage
    ~~~
    From Tyukavina et al. (in review) "Options for global sampling of geographic data"
    Appendix, equations 21, 23, 29-31, 33-35
    """
    #Create a copy of columns "nh", "Ah","ai", "SumPixareaSq" and new columns computed as (equation 23):
    
    #"Correct", the proportion of the sample unit that is correctly classified as target class:
    # min ("Map", "Reference"), and multiplied by the sample unit area; 
    #"Ref": "Reference", multiplied by the sample unit area.
    df1 = pd.concat([df['Stratum'],df['nh'],df['Ah'],df['ai'],df['SumPixareaSq'], pd.Series((np.minimum(df['Map'],df['Reference']) * df['ai']) , name = 'Correct'),
                    pd.Series((df['Reference'] * df['ai']) , name = 'Ref')],axis = 1)
    
    #Equation 21 Compute inclusion probability for each sampled unit
    df1['PIi'] = df1['nh'] * df1['ai'] / df1['Ah']
    
    #Equation 30
    West = (df1.Correct / df1.PIi).sum()
    #Equation 31
    Zest = (df1.Ref / df1.PIi).sum()
    #Equation 29
    PA = West / Zest
    
    #Equation 35
    ByStratum = df1.groupby(by = ['Stratum'])
    K = (ByStratum.SumPixareaSq.median() * ByStratum.nh.median()) /(ByStratum.Ah.median())**2
    
    n = len(df1)
    
    PIi = np.broadcast_to(df1.PIi, (n, n))
    PIj = PIi.T
    Stratumh = np.broadcast_to(df1.Stratum, (n, n))
    nh = np.broadcast_to(df1.nh, (n, n)) 
    Kh = np.broadcast_to(K[df1['Stratum']], (n, n))
    PIiPIj = PIi * PIj
    samestratum = (Stratumh == Stratumh.T).astype(int) #to check whether pixels i and j are in the same stratum
    
    # Equation 34
    PIij_num = (nh - 1) * PIiPIj
    PIij_denom = nh - PIi - PIj + Kh
    PIij = PIij_num / PIij_denom
    
    # Equation 33
    var_i = (df1['Correct'] - PA * df1['Ref']) / df1['PIi']
    var_ij = (1 - PIiPIj / PIij) * np.tensordot(var_i, var_i, axes=0) * samestratum
    var_ij[np.diag_indices_from(var_ij)] = (1 - df1['PIi']) * var_i ** 2
    var = np.sum(var_ij) / Zest**2
    
    SE = np.sqrt(var) * 100
     
    return SE

In [30]:
estimate_PA_SE_without_replacement(data)

1.3788942976134728

#### Functions to estimate % of class1 from class2

In [31]:
def estimate_class_percent (df: pd.DataFrame) -> float:
    """ 
    Function to estimate percent of class1 from class2, both estimated from the sample
    for sampling of units with inclusion probability weighted by the unit area.
    Valid for sampling both with and without replacement.
    ~~~ 
    Input dataframe with number of lines equal the number of sample pixels/polygons,
    and the following columns:
    "Class1" (0-1 - proportion of the sample pixel/polygon identified as class 1 (percentage numerator))
    "Class2" (0-1 - proportion of the sample pixel/polygon identified as class 1 (percentage denumerator))
    "Ah" (stratum area, km²<or any other area unit>)
    "nh" (number of sample pixels in stratum h)
    "ai" (area of sampled pixel in the same units as Ah)
    ~~~
    Returns estimated percent of class1 from the area of class2
    ~~~
    From Tyukavina et al. (in review) "Options for global sampling of geographic data"
    Appendix, equations 21, 22, 29-31
    """
    #Create a copy of columns "nh", "Ah","ai" and new columns "Class1" and "Class2" computed as (equation 23):
    #original columns "Class1" and "Class2" multiplied by the sample unit area;
    df1 = pd.concat([df['nh'],df['Ah'],df['ai'], pd.Series((df['Class1'] * df['ai']), name = 'Class1'),
                    pd.Series((df['Class2'] * df['ai']) , name = 'Class2')],axis = 1)
    
    #Equation 21 Compute inclusion probability for each sampled pixel
    df1['PIi'] = df1['nh'] * df1['ai'] / df1['Ah']
    
    #Equation 30
    West = (df1.Class1 / df1.PIi).sum()
    #Equation 31
    Zest = (df1.Class2 / df1.PIi).sum()
    
    #Equation 29
    PERC = West / Zest * 100

    return PERC

In [32]:
def estimate_class_percent_SE_with_replacement(df: pd.DataFrame) -> float:
    """ 
    Function to estimate the standard error (SE) of percent of class1 from class2, both estimated from the sample
    for sampling of units with inclusion probability weighted by the unit area.
    Valid for sampling with replacement.
    ~~~
    Input dataframe with number of lines equal the number of sample pixels/polygons,
    and the following columns:
    "Class1" (0-1 - proportion of the sample pixel/polygon identified as class 1 (percentage numerator))
    "Class2" (0-1 - proportion of the sample pixel/polygon identified as class 1 (percentage denumerator))
    "Ah" (stratum area, km²<or any other area unit>)
    "nh" (number of sample pixels in stratum h)
    "ai" (area of sampled pixel in the same units as Ah)
    ~~~
    Returns estimated SE of percent of class1 from class2
    ~~~
    From Tyukavina et al. (in review) "Options for global sampling of geographic data"
    Appendix, equations 21, 23, 29-32
    """
    #Create a copy of columns "nh", "Ah","ai" and new columns "Class1" and "Class2" computed as (equation 23):
    #original columns "Class1" and "Class2" multiplied by the sample unit area;
    df1 = pd.concat([df['nh'],df['Ah'],df['ai'], pd.Series((df['Class1'] * df['ai']), name = 'Class1'),
                    pd.Series((df['Class2'] * df['ai']) , name = 'Class2')],axis = 1)
    
    #Equation 21 Compute inclusion probability for each sampled pixel
    df1['PIi'] = df1['nh'] * df1['ai'] / df1['Ah']
    
    #Equation 30
    West = (df1.Class1 / df1.PIi).sum()
    #Equation 31
    Zest = (df1.Class2 / df1.PIi).sum()
    #Equation 29
    PERC = West / Zest
    
    # Equation 32
    var_i = (1 - df1['PIi']) * (df1['Class1'] - PERC * df1['Class2']) * (df1['Class1'] - PERC * df1['Class2']) / (df1['PIi'] * df1['PIi'])
    var = np.sum(var_i) / Zest**2
    
    SE = np.sqrt(var) * 100
     
    return SE

In [33]:
def estimate_class_percent_SE_without_replacement (df: pd.DataFrame) -> float:
    """ 
    Function to estimate the standard error (SE) of percent of class1 from class2, both estimated from the sample
    for sampling of units with inclusion probability weighted by the unit area.
    Valid for sampling without replacement.
    ~~~
    Input dataframe with number of lines equal the number of sample pixels/polygons,
    and the following columns:
    "Stratum" (strata IDs 1 - nstrata)
    "Class1" (0-1 - proportion of the sample pixel/polygon identified as class 1 (percentage numerator))
    "Class2" (0-1 - proportion of the sample pixel/polygon identified as class 1 (percentage denumerator))
    "Ah" (stratum area, km²<or any other area unit>)
    "nh" (number of sample pixels in stratum h)
    "ai" (area of sampled pixel in the same units as Ah)
    "SumPixareaSq" (sum of squared areas of all (not only sampled) units (pixels/polygons) within each stratum)
    ~~~
    Returns estimated SE of percent of class1 from class2
    ~~~
    From Tyukavina et al. (in review) "Options for global sampling of geographic data"
    Appendix, equations 21, 23, 29-31, 33-35
    """
    #Create a copy of columns "Stratum", nh", "Ah","ai", "SumPixareaSq" and new columns "Class1" and "Class2" computed as (equation 23):
    #original columns "Class1" and "Class2" multiplied by the sample unit area;
    df1 = pd.concat([df['Stratum'],df['nh'],df['Ah'],df['ai'],df['SumPixareaSq'], pd.Series((df['Class1'] * df['ai']), name = 'Class1'),
                    pd.Series((df['Class2'] * df['ai']) , name = 'Class2')],axis = 1)
    
    #Equation 21 Compute inclusion probability for each sampled pixel
    df1['PIi'] = df1['nh'] * df1['ai'] / df1['Ah']
    
    #Equation 30
    West = (df1.Class1 / df1.PIi).sum()
    #Equation 31
    Zest = (df1.Class2 / df1.PIi).sum()
    #Equation 29
    PERC = West / Zest
    
    #Equation 35
    ByStratum = df1.groupby(by = ['Stratum'])
    K = (ByStratum.SumPixareaSq.median() * ByStratum.nh.median()) / (ByStratum.Ah.median())**2
    
    n = len(df1)
    
    PIi = np.broadcast_to(df1.PIi, (n, n))
    PIj = PIi.T
    Stratumh = np.broadcast_to(df1.Stratum, (n, n))
    nh = np.broadcast_to(df1.nh, (n, n)) 
    Kh = np.broadcast_to(K[df1['Stratum']], (n, n))
    PIiPIj = PIi * PIj
    samestratum = (Stratumh == Stratumh.T).astype(int) #to check whether pixels i and j are in the same stratum
    
    # Equation 34
    PIij_num = (nh - 1) * PIiPIj
    PIij_denom = nh - PIi - PIj + Kh
    PIij = PIij_num / PIij_denom
    
    # Equation 33
    var_i = (df1['Class1'] - PERC * df1['Class2']) / df1['PIi']
    var_ij = (1 - PIiPIj / PIij) * np.tensordot(var_i, var_i, axes=0) * samestratum
    var_ij[np.diag_indices_from(var_ij)] = (1 - df1['PIi']) * var_i ** 2
    var = np.sum(var_ij) / Zest**2
    
    SE = np.sqrt(var) * 100
     
    return SE

In [34]:
#Example of use: estimate % of Types 0-3 from the overall area of the target class (Reference>0).

functions = [estimate_class_percent, estimate_class_percent_SE_with_replacement, estimate_class_percent_SE_without_replacement]
names = ['% from target class','SE with replacement', 'SE without replacement']
results = pd.DataFrame()
datacopy = pd.DataFrame()

for classtype in data['RefType'].unique():
    datacopy = data.copy()
    #A new Class1 column, where reference values are set to zero if class type is different from the current class type
    datacopy['Class1'] = np.where(datacopy['RefType'] == classtype, datacopy['Reference'], 0)
    
    #Set Class2 equal to Reference (total area of target class)
    datacopy['Class2'] = datacopy['Reference']
    
    
    values = {}
    values = {nm:[fn(datacopy)] for fn, nm in zip(functions,names)}
    values["Estimate"] = classtype
    values_pd = pd.DataFrame(values).set_index("Estimate")
    results = pd.concat([values_pd, results])

results = results.reset_index()

results.style.hide_index().format({name: '{:.2f}' for name in names})


Estimate,% from target class,SE with replacement,SE without replacement
Type3,31.30,2.61,2.61
Type2,49.69,2.80,2.81
Type0,0.00,0.00,0.00
Type1,19.01,2.19,2.20
